In [24]:
#fine-tuning a multiclass text classification task using the DistilBERT-base-uncased model from the Transformers library and saving it i

In [1]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments

In [2]:
import pandas as pd
import tensorflow as tf
from transformers import DistilBertTokenizerFast, TFDistilBertForSequenceClassification
from sklearn.model_selection import train_test_split


In [5]:
df = pd.read_csv("../0Round2TrainingSI/SI_1298LDataset.csv")
#replace the dataset path by your data

In [6]:
df.head()

,post,si
0,’ self harm year ’ look tempt ’ fuck alone fee...,1
1,anybody suggest overdose work someone try work...,1
2,blood run vein every single cell body even bra...,1
3,isnt first time ive felt like never felt final...,1
4,ive beenwaiting moment year everyone asleeo im...,1


In [7]:
df.shape

(1298, 2)

In [8]:
df_silver2 = pd.read_csv("Round3_LabeledData2020_Head600.csv")

In [9]:
df_silver2.head()

,Unnamed: 0,author,created_utc,selftext,suicideIdeation
0,6,trash_account173638,2020-03-01 00:02:43,recent diagnos sever ocd mind constant assault...,1
1,17,Bad_Operator,2020-03-01 00:07:16,wrote note decid slit wrist sit tub sorri son,0
2,38,Bbrave1,2020-03-01 00:16:12,slept poor last night awok terribl exhaust moo...,1
3,56,kirin113,2020-03-01 00:23:19,hi new reddit sure right place post wonder any...,1
4,87,stonedfordepression,2020-03-01 00:35:41,struggl mental health issu sinc kid attempt su...,1


In [10]:
df_silver2.shape

(538, 5)

In [11]:
# Select specific columns from df_silver and rename them
df_silver2 = df_silver2[['selftext', 'suicideIdeation']]
df_silver2.columns = ['post', 'si']
print(df_silver2.shape)

(538, 2)


In [12]:

# Select specific columns from df
df = df[['post', 'si']]
df.columns = ['post', 'si']
df.shape

(1298, 2)

In [13]:
df_silver2.shape

(538, 2)

In [15]:
# Concatenate the dataframes
df_1800 = pd.concat([df_silver2, df], ignore_index=True)


In [17]:
df_1800.shape

(1836, 2)

In [19]:
# Save the new dataframe to CSV
df_1800.to_csv("SI_Round3_1836_LDataset.csv", index=False)

In [3]:
df = pd.read_csv("SI_Round3_1836_LDataset.csv")

In [4]:
df.shape

(1836, 2)

In [5]:
df.head(10)

,post,si
0,recent diagnos sever ocd mind constant assault...,1
1,wrote note decid slit wrist sit tub sorri son,0
2,slept poor last night awok terribl exhaust moo...,1
3,hi new reddit sure right place post wonder any...,1
4,struggl mental health issu sinc kid attempt su...,1
5,everyth life slowli gotten wors wors use go un...,0
6,serious contempl suicid enjoy life want contin...,1
7,go ignor peopl tell know research pleas let know,0
8,pleas know painless relat simpl way commit sui...,1
9,went first therapi session last tuesday resche...,0


In [6]:
import pandas as pd

# Assuming you have the dataset stored in a pandas DataFrame called 'data'
# You can modify the code accordingly based on your specific dataset structure

suicide_count = 0
for index, row in df.iterrows():
    if row['si'] == 1:
        suicide_count += 1

print("Number of posts with a suicide label of 1:", suicide_count)


Number of posts with a suicide label of 1: 1035


In [7]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import contractions
def clean_text(text):
    # Replace missing values with empty strings
    text = str(text)
    # Convert to lowercase
    text = text.lower()
    #fix the contraction 
    text = contractions.fix(text)
    # Remove URLs
    text = re.sub(r'http\S+', '', text)

    # Remove mentions and hashtags
    text = re.sub(r'@\w+|#\w+', '', text)

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Tokenize text
    tokens = nltk.word_tokenize(text)

    # Lowercase text
    tokens = [token.lower() for token in tokens]

    #Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Stem words
    stemmer = SnowballStemmer('english')
    tokens = [stemmer.stem(token) for token in tokens]

    # Join tokens into a string
    text = ' '.join(tokens)
    
    return(text)


In [8]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /home/jzhu10/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [9]:
# Clean the 'post' column
df['cleaned_post'] = df['post'].apply(lambda x: clean_text(x) if isinstance(x, str) else x)

In [10]:
df.tail(3)

,post,si,cleaned_post
1833,['That sounds really weird.Maybe you were Dist...,0,sound realli weirdmayb distract someth sound h...
1834,['Dont know there as dumb as it sounds I feel ...,1,know dumb sound feel hyperact behavior even de...
1835,"['&gt;It gets better, trust me.Ive spent long ...",0,gtit get better trust mei spent long enough bl...


In [11]:
# Split the data into train and validation sets
# Split the data into train and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(df["cleaned_post"], df["si"], test_size=0.2, random_state=42)


In [13]:
# Load the tokenizer and encode the texts
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True)
test_encodings = tokenizer(test_texts.tolist(), truncation=True, padding=True)


In [14]:
# Convert the data to tf.data.Dataset format
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), test_labels))


In [32]:
# Define the model
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=4e-5),
              loss=model.compute_loss,
              metrics=['accuracy'])
# Train the model
model.fit(train_dataset.shuffle(len(train_dataset)).batch(16),
          epochs=10,
          batch_size=16,
          validation_data=test_dataset.shuffle(len(test_dataset)).batch(16))
import numpy as np
# Evaluate the model on test data
preds = model.predict(test_dataset.batch(16))
pred_labels = np.argmax(preds.logits, axis=1)

# Calculate the accuracy
accuracy = np.mean(pred_labels == test_labels)
print("Accuracy:", accuracy)

# Save the model
model.save_pretrained('./1300Model1')
tokenizer.save_pretrained('./1300Model1')

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'activation_13', 'vocab_transform', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'classifier', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use i

Epoch 1/10


/home/jzhu10/.conda/envs/twitter/lib/python3.6/site-packages/tensorflow/python/autograph/impl/api.py:376: FutureWarning: The old compute_loss method is deprecated as it conflicts with the Keras compute_loss method added in TF 2.8. If you want the original HF compute_loss, please call hf_compute_loss() instead. From TF versions >= 2.8, or Transformers versions >= 5, calling compute_loss() will get the Keras method instead.
  return py_builtins.overload_of(f)(*args)


92/92 [==============================] - 369s 4s/step - loss: 0.6803 - accuracy: 0.5702 - val_loss: 0.6297 - val_accuracy: 0.6467
Epoch 2/10
92/92 [==============================] - 362s 4s/step - loss: 0.6098 - accuracy: 0.6662 - val_loss: 0.5942 - val_accuracy: 0.7038
Epoch 3/10
92/92 [==============================] - 353s 4s/step - loss: 0.4822 - accuracy: 0.7643 - val_loss: 0.5555 - val_accuracy: 0.7283
Epoch 4/10
92/92 [==============================] - 351s 4s/step - loss: 0.3397 - accuracy: 0.8549 - val_loss: 0.7166 - val_accuracy: 0.6957
Epoch 5/10
92/92 [==============================] - 351s 4s/step - loss: 0.1918 - accuracy: 0.9203 - val_loss: 0.8412 - val_accuracy: 0.7011
Epoch 6/10
92/92 [==============================] - 352s 4s/step - loss: 0.1148 - accuracy: 0.9557 - val_loss: 0.9383 - val_accuracy: 0.6875
Epoch 7/10
92/92 [==============================] - 352s 4s/step - loss: 0.0833 - accuracy: 0.9714 - val_loss: 1.0958 - val_accuracy: 0.6739
Epoch 8/10
92/92 [======

('./1300Model1/tokenizer_config.json',
 './1300Model1/special_tokens_map.json',
 './1300Model1/vocab.txt',
 './1300Model1/added_tokens.json',
 './1300Model1/tokenizer.json')

In [33]:
# Define the model
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
              loss=model.compute_loss,
              metrics=['accuracy'])
# Train the model
model.fit(train_dataset.shuffle(len(train_dataset)).batch(16),
          epochs=10,
          batch_size=32,
          validation_data=test_dataset.shuffle(len(test_dataset)).batch(16))
import numpy as np
# Evaluate the model on test data
preds = model.predict(test_dataset.batch(16))
pred_labels = np.argmax(preds.logits, axis=1)

# Calculate the accuracy
accuracy = np.mean(pred_labels == test_labels)
print("Accuracy:", accuracy)

# Save the model
model.save_pretrained('./1800Model2')
tokenizer.save_pretrained('./1800Model2')

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'activation_13', 'vocab_transform', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'classifier', 'dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use i

Epoch 1/10


/home/jzhu10/.conda/envs/twitter/lib/python3.6/site-packages/tensorflow/python/autograph/impl/api.py:376: FutureWarning: The old compute_loss method is deprecated as it conflicts with the Keras compute_loss method added in TF 2.8. If you want the original HF compute_loss, please call hf_compute_loss() instead. From TF versions >= 2.8, or Transformers versions >= 5, calling compute_loss() will get the Keras method instead.
  return py_builtins.overload_of(f)(*args)


92/92 [==============================] - 357s 4s/step - loss: 0.6806 - accuracy: 0.5817 - val_loss: 0.6441 - val_accuracy: 0.6495
Epoch 2/10
92/92 [==============================] - 352s 4s/step - loss: 0.6236 - accuracy: 0.6594 - val_loss: 0.5510 - val_accuracy: 0.6929
Epoch 3/10
92/92 [==============================] - 354s 4s/step - loss: 0.4975 - accuracy: 0.7568 - val_loss: 0.5403 - val_accuracy: 0.7092
Epoch 4/10
92/92 [==============================] - 355s 4s/step - loss: 0.3608 - accuracy: 0.8399 - val_loss: 0.8004 - val_accuracy: 0.6332
Epoch 5/10
92/92 [==============================] - 355s 4s/step - loss: 0.2404 - accuracy: 0.8971 - val_loss: 0.7360 - val_accuracy: 0.6821
Epoch 6/10
92/92 [==============================] - 351s 4s/step - loss: 0.1244 - accuracy: 0.9659 - val_loss: 1.0613 - val_accuracy: 0.6821
Epoch 7/10
92/92 [==============================] - 349s 4s/step - loss: 0.0740 - accuracy: 0.9734 - val_loss: 1.0451 - val_accuracy: 0.7092
Epoch 8/10
92/92 [======

('./1800Model2/tokenizer_config.json',
 './1800Model2/special_tokens_map.json',
 './1800Model2/vocab.txt',
 './1800Model2/added_tokens.json',
 './1800Model2/tokenizer.json')

In [34]:
# Define the model
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
              loss=model.compute_loss,
              metrics=['accuracy'])
# Train the model
model.fit(train_dataset.shuffle(len(train_dataset)).batch(16),
          epochs=20,
          batch_size=32,
          validation_data=test_dataset.shuffle(len(test_dataset)).batch(16))

# Evaluate the model
loss, accuracy = model.evaluate(test_dataset.batch(16))

print(f"Loss: {loss}, Accuracy: {accuracy}")
# Save the model
model.save_pretrained('./1800Model3')
tokenizer.save_pretrained('./1800Model3')

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'activation_13', 'vocab_transform', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_59']
You should probably TRAIN this model on a down-stream task to be able to use i

Epoch 1/20


/home/jzhu10/.conda/envs/twitter/lib/python3.6/site-packages/tensorflow/python/autograph/impl/api.py:376: FutureWarning: The old compute_loss method is deprecated as it conflicts with the Keras compute_loss method added in TF 2.8. If you want the original HF compute_loss, please call hf_compute_loss() instead. From TF versions >= 2.8, or Transformers versions >= 5, calling compute_loss() will get the Keras method instead.
  return py_builtins.overload_of(f)(*args)


92/92 [==============================] - 359s 4s/step - loss: 0.6787 - accuracy: 0.5688 - val_loss: 0.6333 - val_accuracy: 0.6386
Epoch 2/20
92/92 [==============================] - 353s 4s/step - loss: 0.6096 - accuracy: 0.6805 - val_loss: 0.5446 - val_accuracy: 0.7391
Epoch 3/20
92/92 [==============================] - 345s 4s/step - loss: 0.4965 - accuracy: 0.7568 - val_loss: 0.5639 - val_accuracy: 0.6957
Epoch 4/20
92/92 [==============================] - 351s 4s/step - loss: 0.3826 - accuracy: 0.8283 - val_loss: 0.6474 - val_accuracy: 0.6902
Epoch 5/20
92/92 [==============================] - 356s 4s/step - loss: 0.2354 - accuracy: 0.8992 - val_loss: 0.8070 - val_accuracy: 0.6848
Epoch 6/20
92/92 [==============================] - 353s 4s/step - loss: 0.1355 - accuracy: 0.9503 - val_loss: 1.0562 - val_accuracy: 0.7011
Epoch 7/20
92/92 [==============================] - 350s 4s/step - loss: 0.1156 - accuracy: 0.9591 - val_loss: 1.1792 - val_accuracy: 0.6984
Epoch 8/20
92/92 [======

('./1800Model3/tokenizer_config.json',
 './1800Model3/special_tokens_map.json',
 './1800Model3/vocab.txt',
 './1800Model3/added_tokens.json',
 './1800Model3/tokenizer.json')

In [35]:
# Define the model
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
# Compile the model

# Compile the model with adjusted learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss=model.compute_loss,
              metrics=['accuracy'])
# Train the model with increased epochs
model.fit(train_dataset.shuffle(len(train_dataset)).batch(16),
          epochs=30,
          batch_size=32,
          validation_data=test_dataset.shuffle(len(test_dataset)).batch(16))

# Evaluate the model
loss, accuracy = model.evaluate(test_dataset.batch(16))

print(f"Loss: {loss}, Accuracy: {accuracy}")
# Save the model
model.save_pretrained('./1800Model1_7')
tokenizer.save_pretrained('./1800Model1_7')

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'activation_13', 'vocab_transform', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_79', 'pre_classifier', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

Epoch 1/30
92/92 [==============================] - 358s 4s/step - loss: 0.6733 - accuracy: 0.5920 - val_loss: 0.6554 - val_accuracy: 0.6630
Epoch 2/30
92/92 [==============================] - 351s 4s/step - loss: 0.6350 - accuracy: 0.6546 - val_loss: 0.6485 - val_accuracy: 0.6223
Epoch 3/30
92/92 [==============================] - 350s 4s/step - loss: 0.5436 - accuracy: 0.7316 - val_loss: 0.6161 - val_accuracy: 0.7011
Epoch 4/30
92/92 [==============================] - 353s 4s/step - loss: 0.4418 - accuracy: 0.8052 - val_loss: 0.6319 - val_accuracy: 0.6658
Epoch 5/30
92/92 [==============================] - 352s 4s/step - loss: 0.3238 - accuracy: 0.8699 - val_loss: 0.6385 - val_accuracy: 0.7011
Epoch 6/30
92/92 [==============================] - 349s 4s/step - loss: 0.2114 - accuracy: 0.9285 - val_loss: 0.8045 - val_accuracy: 0.6929
Epoch 7/30
92/92 [==============================] - 351s 4s/step - loss: 0.1331 - accuracy: 0.9544 - val_loss: 0.9431 - val_accuracy: 0.6929
Epoch 8/30
92

('./1800Model1_7/tokenizer_config.json',
 './1800Model1_7/special_tokens_map.json',
 './1800Model1_7/vocab.txt',
 './1800Model1_7/added_tokens.json',
 './1800Model1_7/tokenizer.json')

In [18]:
import tensorflow as tf
from transformers import TFDistilBertForSequenceClassification, DistilBertTokenizerFast
from tensorflow.keras.callbacks import ModelCheckpoint

# Load the tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# Define the model architecture
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Get the last transformer layer
last_layer = model.distilbert.transformer.layer[-1]

# Add a dropout layer
last_layer.attention_dropout = tf.keras.layers.Dropout(0.1)
last_layer.output_dropout = tf.keras.layers.Dropout(0.1)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=4e-5),
              loss=model.compute_loss,
              metrics=['accuracy'])

# Define the ModelCheckpoint callback to save the best model weights
checkpoint_callback = ModelCheckpoint(filepath='best_model_weights.h5',
                                      save_best_only=True,
                                      save_weights_only=True,
                                      monitor='val_accuracy',
                                      mode='max')

# Train the model with the ModelCheckpoint callback and adjusted parameters
model.fit(train_dataset.shuffle(len(train_dataset)).batch(32),
          epochs=50,
          batch_size=64,
          validation_data=test_dataset.shuffle(len(test_dataset)).batch(64),
          callbacks=[checkpoint_callback])

# Load the weights of the model with the highest validation accuracy
model.load_weights('best_model_weights.h5')

# Evaluate the model on the test dataset
loss, accuracy = model.evaluate(test_dataset.batch(64))

# Print the highest accuracy and loss achieved during training
print(f"Highest Accuracy: {checkpoint_callback.best} - Loss: {loss}")

# Save the model
model.save_pretrained('./1800Model1_9')
tokenizer.save_pretrained('./1800Model1_9')

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_transform', 'vocab_projector', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_63']
You should probably TRAIN this model on a down-stream task to be able to use i

Epoch 1/50
46/46 [==============================] - 411s 9s/step - loss: 0.6802 - accuracy: 0.5579 - val_loss: 0.6639 - val_accuracy: 0.5625
Epoch 2/50
46/46 [==============================] - 413s 9s/step - loss: 0.6189 - accuracy: 0.6553 - val_loss: 0.5832 - val_accuracy: 0.6766
Epoch 3/50
46/46 [==============================] - 406s 9s/step - loss: 0.5215 - accuracy: 0.7302 - val_loss: 0.5656 - val_accuracy: 0.7065
Epoch 4/50
46/46 [==============================] - 404s 9s/step - loss: 0.3818 - accuracy: 0.8324 - val_loss: 0.6486 - val_accuracy: 0.6957
Epoch 5/50
46/46 [==============================] - 401s 9s/step - loss: 0.2216 - accuracy: 0.9128 - val_loss: 0.7474 - val_accuracy: 0.7092
Epoch 6/50
46/46 [==============================] - 401s 9s/step - loss: 0.1364 - accuracy: 0.9455 - val_loss: 1.0343 - val_accuracy: 0.6848
Epoch 7/50
46/46 [==============================] - 399s 9s/step - loss: 0.1375 - accuracy: 0.9462 - val_loss: 0.9602 - val_accuracy: 0.7011
Epoch 8/50
46

('./1800Model1_9/tokenizer_config.json',
 './1800Model1_9/special_tokens_map.json',
 './1800Model1_9/vocab.txt',
 './1800Model1_9/added_tokens.json',
 './1800Model1_9/tokenizer.json')